#### Bibliotecas e configuracoes

In [1]:
import os
import json
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import googlemaps
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

# Pacote com todas as funcoes devenvolvidas para esse projeto
from functions_proj_scraping import *

# Configs
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.precision', 2)
warnings.filterwarnings('ignore')

#### Pegar todos os anuncios de todas as paginas do site (macro) com paralelismo

In [ ]:
url = 'https://www.lagoimobiliaria.com.br/pesquisa-de-imoveis/?locacao_venda=V&id_cidade%5B%5D=90&id_tipo_imovel%5B%5D=5&id_tipo_imovel%5B%5D=20&id_tipo_imovel%5B%5D=19&finalidade=residencial&dormitorio=0&garagem=0&vmi=&vma='
#url = 'https://www.lagoimobiliaria.com.br/pesquisa-de-imoveis/?locacao_venda=V&id_cidade%5B%5D=90&id_tipo_imovel%5B%5D=4&finalidade=0&dormitorio=0&garagem=0&vmi=&vma='
paginas = total_paginas_site(url)

base_geral = []

# Funcao para executar o scrapping
def processar_pagina(pag):
    soup = gerando_soup_principal(url, pag)
    pagina = soup.find('div', {'class':'properties_item_two row'}).findAll('div', class_='col-lg-3 col-md-4 col-sm-6 col-xs-12')
    
    resultados_pagina = []
    
    for item in range(0, len(pagina)):
        resultado = dados_pag_principal(pagina[item], item)
        resultados_pagina.append(resultado)
    
    return resultados_pagina

# Paralelismo
with concurrent.futures.ThreadPoolExecutor() as executor:
    resultados_paginas = list(executor.map(processar_pagina, range(paginas)))

# "Aplanhando" a lista de listas para obter uma única lista de resultados
base_geral = [resultado for resultados_pagina in resultados_paginas for resultado in resultados_pagina]

#### Carregando dados macro para dataframe e tratando dados

In [ ]:
# Criando dataframe e Trocando Nan -> 0 e removendo duplicidades
df = pd.DataFrame(base_geral)
df.fillna(0, inplace=True)
df.drop_duplicates(subset=['link'], keep='first', inplace=True, ignore_index=True)

# Criando coluna com nro de indice para merge posterior
df['indice'] = df.index

# Corrigindo encoding no link para evitar erros
df['link'] = df.apply(lambda row: safeStr(row['link']), axis=1)

# Reindexando
df.reset_index(inplace=True, drop=True)

#### Pegando dados complementares de todos os anuncios com paralelismo

In [ ]:
# Funcao para processar o scrapping de cada anuncio especificamente
def processar_anuncio(link, index):
    try:
        pagina = gerando_soup_anuncio(link)
        resultado = dados_pag_anuncio(pagina, index)
        if resultado != 'erro':
            return resultado
    except:
        pass
    return None

# Criando df com links para loop no paralelismo
links = pd.DataFrame(df['link'])

# Criando lista para gravar resultados
base_geral = []

# Usando paralelismo com limite de operacoes simultaneas
max_workers = os.cpu_count()
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(processar_anuncio, row['link'], index): index for index, row in links.iterrows()}

    # Coletando  e gravando os resultados na lista
    for future in concurrent.futures.as_completed(futures):
        resultado = future.result()
        if resultado:
            base_geral.append(resultado)

#### Carregando dados anuncios para dataframe, tratando dados e combinando bases

In [ ]:
# Criando dataframe e trocando Nan -> 0
df2 = pd.DataFrame(base_geral) 
df2.fillna(0, inplace=True)

# Mesclando df + df2
base_final = df.merge(df2, how='inner', on='indice')
base_final.fillna(0, inplace=True)
#del df, df2

#### Tratando base final

In [ ]:
# Criando coluna de preco
# Existem 2 campos de precos no site e ainda a possibilidade de haver apenas preco de aluguel
# Regras aqui: Se valor >= valor_venda entao mantem valor, senao valor_venda
base_final['preco'] = np.where(base_final['valor']>=base_final['valor_venda'],
                               base_final['valor'],
                               base_final['valor_venda'])

# Regras aqui: Se preco == valor_aluguel entao zera valor (queremos apenas anuncios de venda), senao mantem preco. 
base_final['preco'] = np.where(base_final['preco']==base_final['valor_aluguel'],
                               0,
                               base_final['preco'])

# Arredondando centavos
base_final['preco'] = base_final['preco'].round(0)

# Criando faixa de valores
faixas = ['até 250k','250k a 500k','501k a 750k','751k a 1M','acima 1M']
base_final['faixas_precos'] = pd.cut(base_final['preco'], bins=[0,250000,500000,750000,1000000,8000000], labels=faixas)

# Criando colunas de metragens
# Para CASAS existe area total e terreno, para APTOS existe area util e area construida
base_final['area_total_ou_terreno'] = np.where(base_final['area_total']==0,
                                               base_final['area_terreno'],
                                               base_final['area_total'])

base_final['area_util_ou_construida'] = np.where(base_final['area_util']==0,
                                                 base_final['area_construida'],
                                                 base_final['area_util'])

base_final['check_area'] = base_final['area_total_ou_terreno'] + base_final['area_util_ou_construida']

# Excluindo valores zerados
base_final = base_final[base_final['preco'] > 0]
base_final = base_final[base_final['check_area'] > 0]
base_final = base_final[base_final['classificacao'] != '0']
base_final.drop(columns=['indice','valor_venda','valor','area_util','area_total','check_area',
                         'area_terreno','area_construida'], inplace=True)

# Reindexando
base_final.reset_index(inplace=True, drop=True)

# Convertendo ints
colunas = ['quartos','suites','banheiros','vagas']
base_final[colunas] = base_final[colunas].astype('int32')

# Removendo palavras sem contexto do campo bairros e zona
base_final['bairro'] = base_final['bairro'].str.replace('lancamento ', '')
base_final['zona'] = base_final['zona'].str.replace('zona ', '')

# Convertendo objetos para categorias
colunas = ['bairro','cidade','zona','classificacao','pagina']
base_final[colunas] = base_final[colunas].astype('category')

# Ajustando bytes de tipagens numericas
base_final = tipagem(base_final)

# Remocao de outliers
base_final = remove_outliers(base_final)

# Criando coluna de preco x metragem
base_final['preco_m2'] = [round((preco / area_total),0) if area_total > area_util else round((preco / area_util),0) 
                          for preco, area_total, area_util in zip(base_final['preco'], 
                                                                  base_final['area_total_ou_terreno'], 
                                                                  base_final['area_util_ou_construida'])]

# Adicionando coluna para simplificar contagens
base_final['registros'] = 1

# Reorganizando
base_final = base_final[['bairro','cidade','zona','descricao','telefones','link','link_foto_capa','classificacao','pagina',
                         'faixas_precos', 'preco','preco_m2','quartos','suites','banheiros','vagas','itens_do_imovel', 
                         'itens_do_condominio','area_total_ou_terreno','area_util_ou_construida','valor_condominio', 
                         'valor_iptu','valor_aluguel','registros']]

# Criando ID
base_final['id'] = base_final.index

# Reindexando
base_final.reset_index(inplace=True, drop=True)

#### Adicionando coordenadas dos bairros com API Google Maps

In [ ]:
# Iniciando API google maps
api_key = json.load(open(r'C:\Users\Valeria\Projetos_DS\cred_gclp.json')).get('api_key')
gmaps = googlemaps.Client(key=api_key)

def coletar_coordenadas(local):
    geocode = gmaps.geocode(local)
    lat = geocode[0]['geometry']['location']['lat']
    lng = geocode[0]['geometry']['location']['lng']

    return [lat,lng]

In [ ]:
base_bairros = base_final[['bairro','cidade','zona']].drop_duplicates(keep='first').reset_index(drop=True)

for linha in range(0,len(base_bairros)):
    local = base_bairros.loc[linha, 'bairro'] +', '+ base_bairros.loc[linha, 'cidade']
    lat_lng = coletar_coordenadas(local)
    
    base_bairros.loc[linha, 'lat'] = lat_lng[0]
    base_bairros.loc[linha, 'lng'] = lat_lng[1]

#### Trabalhando base de itens do imovel

In [18]:
# Quebrando lista de itens da base_final
base_itens_imovel = separa_itens(base_final[['id','itens_do_imovel']],'itens_do_imovel')

# Tratando dados
base_itens_imovel['itens'] = base_itens_imovel['itens'].str.replace("'",'').str.strip()

# Excluindo informacoes pouco uteis
lista_exclusao = ['apartamento por andar', 'torre', 'terre', 'quantidade de andares', 'restricao']
lista_indices = []
for item, index in zip(base_itens_imovel['itens'],base_itens_imovel['id']):
    for item_exclusao in lista_exclusao:
        if item_exclusao in item:
            lista_indices.append(index)
            
base_itens_imovel = base_itens_imovel[~base_itens_imovel['id'].isin(lista_indices)]
base_itens_imovel.reset_index(inplace=True,drop=True)

# Alterando registros (Agrupando similares)
lista_alteracao = ['armario','elevador','piscina','portaria','varanda']

for linha in range(0, len(base_itens_imovel)):
    item_imovel = base_itens_imovel.loc[linha, 'itens']
    for item_alteracao in lista_alteracao:
        if item_alteracao in item_imovel:
            base_itens_imovel.loc[linha, 'itens'] = item_alteracao

In [19]:
# Criando grupos
grupo_cond_seg = ['gas encanado','cerca eletrica','medidor de agua individual','portaria',
                  'cameras de seguranca','portao eletronico','alarme','elevador','gerador']

grupo_lazer = ['piscina','espaco kids','salao de jogos','salao de festa','quadra tenis','quadra poliesportiva',
               'playground', 'campo de futebol','pista de caminhada','espaco gourmet','fitness','cinema',
               'bicicletario','espaco mulher', 'churrasqueira']

grupo_itens_imovel = ['sala de estar', 'armario', 'lavabo', 'box', 'quintal', 'ventilador de teto', 'sacada',
                      'escritorio', 'varanda', 'ar condicionado', 'banheiro de servico', 'mobiliado','sauna',
                      'iluminacao','jacuzzi']

for linha in range(0, len(base_itens_imovel)):
    item_imovel = base_itens_imovel.loc[linha, 'itens']
    if item_imovel in grupo_cond_seg:
        base_itens_imovel.loc[linha, 'grupo'] = 'itens_seg_e_cond'
    elif item_imovel in grupo_lazer:
        base_itens_imovel.loc[linha, 'grupo'] = 'itens_lazer'
    elif item_imovel in grupo_itens_imovel:
        base_itens_imovel.loc[linha, 'grupo'] = 'itens_imovel'
    else:
        base_itens_imovel.loc[linha, 'grupo'] = 'sem_classificacao'

In [20]:
# Adicionando contagem de itens na base_final
contagem_itens = base_itens_imovel.groupby(by=['id','grupo']).count()
contagem_itens = contagem_itens.pivot_table(index='id', columns='grupo', values='itens').fillna(0).reset_index()
base_final = base_final.merge(contagem_itens[['id','itens_imovel','itens_lazer','itens_seg_e_cond']], how='inner', on='id')

# Convertendo ints
colunas = ['itens_imovel','itens_lazer','itens_seg_e_cond']
base_final[colunas] = base_final[colunas].astype('int32')

#### Exportando arquivos csv

In [25]:
base_final.to_csv('bases/base_final_anuncios.csv',sep=';', encoding='utf-8', index=False, header=True)
base_itens_imovel.to_csv('bases/base_itens_imovel.csv',sep=';', encoding='utf-8', index=False, header=True)
base_bairros.to_csv('bases/base_bairros.csv',sep=';', encoding='utf-8', index=False, header=True, decimal='.')

In [2]:
# Importando CSVs para validacao
#base_final = pd.read_csv('bases/base_final_anuncios.csv', sep=';', encoding='utf-8')
#base_final['pagina'] = base_final['pagina'].astype('category')
#base_bairros = pd.read_csv('bases/base_bairros.csv', sep=',', encoding='utf-8')
#base_itens_imovel = pd.read_csv('bases/base_itens_imovel.csv', sep=';', encoding='utf-8')